In [4]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import pickle as pk

/home/aww66/.conda/envs/huggingface/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
# input
checkpoint_dir = "./models/"
checkpoint_name = "mdistilbert-fr"
model_name = 'distilbert-base-multilingual-cased'


data_dir = "./data/"
output_dir = "./data/output/"
device_name = "cuda" if torch.cuda.is_available() else "cpu"
num_labels = 2

print(device_name)

LANGS = ['en', 'de', 'ja', 'es', 'fr', 'zh']

print("init model...")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint_dir + checkpoint_name, 
                                                           num_labels=num_labels).to(device_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

cuda
init model...


In [6]:
from sklearn.metrics import classification_report, roc_auc_score
import scipy.stats as stats

def batch(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

In [7]:
class Evaluator:
    def __init__(self, model, tokenizer, eval_lang):
        self.model = model
        self.model.eval() # set to evaluation mode
        self.tokenizer = tokenizer
        self.eval_lang = eval_lang
        self.text_input = None
        self.true_labels = None
        
        self.load_data()
        self.logits = None
        
    def load_data(self):
        data = pk.load(open(data_dir + f"clean_{self.eval_lang}_test.pk", "rb"))
        self.text_input = data['text']
        self.true_labels = data['binary_labels']
        
    def make_prediction(self):
        logits = None
        with torch.no_grad():
            for batched_inputs in batch(self.text_input, 20):
                encoding = self.tokenizer(batched_inputs, 
                                          truncation=True, 
                                          padding=True, 
                                          return_tensors='pt').to(device_name)
                batched_logits = self.model(**encoding)[0]
                if logits is None:
                    logits = batched_logits
                else:
                    logits = torch.cat((logits, batched_logits), dim=0)
        self.logits = logits
        return logits
    
    def evaluate(self):
        if self.logits is None:
            raise Exception("Run prediction first!")
        _, predicted_labels = torch.max(self.logits, dim=1)
        predicted_labels = predicted_labels.tolist()
        report = classification_report(predicted_labels, self.true_labels, output_dict=True)
        report['kendall-tau'], _ = stats.kendalltau(self.true_labels, self.logits[:, 1].tolist())
        report['auc'] = roc_auc_score(self.true_labels, self.logits[:, 1].tolist())
        return report

In [14]:
pred_scores = dict()
eval_reports = dict()
for lang in LANGS:
    ev = Evaluator(model, tokenizer, lang)
    pred_scores[lang] = ev.make_prediction()
    eval_reports[lang] = ev.evaluate()

In [15]:
torch.save(pred_scores, output_dir + f"{checkpoint_name}_scores.pt")
pk.dump(eval_reports, open(output_dir + f"{checkpoint_name}_eval_reports.pk", "wb"))

## study

In [19]:
import pandas as pd

metrics_df = None
metrics = ['accuracy', 'auc', 'kendall-tau']
for model_lang in ['en', 'fr', 'zh']:
    checkpoint_name = f'mdistilbert-{model_lang}'
    eval_reports = pk.load(open(output_dir + f"{checkpoint_name}_eval_reports.pk", "rb"))
    df = pd.DataFrame.from_dict(eval_reports, orient='index')
    df['model_lang'] = model_lang
    df = df.reset_index().rename({"index": "test_lang"}, axis=1)
    if metrics_df is None:
        metrics_df = df[['model_lang', 'test_lang', *metrics]].copy()
    else:
        metrics_df = pd.concat(
            (
                metrics_df, 
                df[['model_lang', 'test_lang', *metrics]]
            )
        )
        
        

In [20]:
pd.pivot(metrics_df, index='model_lang', 
         columns = 'test_lang', 
         values=metrics)

accuracy                                               auc  \
test_lang        de      en      es      fr      ja      zh        de   
model_lang                                                              
en           0.7116  0.8838  0.7268  0.7620  0.5860  0.7026  0.757209   
fr           0.7394  0.7226  0.7808  0.8856  0.5962  0.6840  0.808079   
zh           0.5848  0.6946  0.5976  0.5094  0.6572  0.8412  0.734220   

                                                             kendall-tau  \
test_lang         en        es        fr        ja        zh          de   
model_lang                                                                 
en          0.951686  0.850235  0.844972  0.731073  0.756772    0.356435   
fr          0.837050  0.873063  0.947676  0.675161  0.735325    0.426929   
zh          0.812382  0.808288  0.760685  0.730695  0.912239    0.324577   

                                                              
test_lang         en        es        fr        ja        zh  
model_lang                                                    
en          0.625938  0.485349  0.478055  0.320217  0.355830  
fr          0.467076  0.516983  0.620380  0.242735  0.326108  
zh          0.432892  0.427219  0.361253  0.319692  0.571272